
Fleet Delivery Optimisation -- Executive Analytics

**The Company** / Supply-Chain Intelligence Group  
**Author:** Pedro Paulo da Cruz Mendes -- AI / Cloud Architect Candidate  
**Date:** February 2026

---

## The Business Problem

The Company runs 15 trucks across 4 vehicle classes (FHWA Class 5 through 8), all dispatching from a single hub in Cincinnati, OH. The fleet serves 300 order lines at 58 delivery sites scattered across North America -- Toronto, Portland, Las Vegas, Albany, and dozens of cities in between.

The problem is straightforward to state and brutal to solve. Manually planned routes waste truck capacity, burn excess fuel, and produce unreliable delivery windows. At $3.75 per gallon of diesel, with FMCSA forcing a 10-hour mandatory rest after every 14 hours of driving, bad routing is expensive.

This analysis answers one question:

> How should The Company assign orders to trucks and sequence deliveries to minimise total fleet cost while respecting vehicle capacities, federal rest regulations, and delivery commitments?

### Formal Problem Definition

In operations research, this is a **Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)**. The mathematical formulation:

$$\min \sum_{k \in K} \sum_{(i,j) \in A} c_{ij} \cdot x_{ijk}$$

subject to:

- Every order is delivered exactly once: $\sum_{k \in K} y_{ik} = 1 \;\forall i \in N$
- No truck exceeds its payload: $\sum_{i \in N} q_i \cdot y_{ik} \leq Q_k \;\forall k \in K$
- Each truck departs from and returns to the depot (flow conservation)
- Work hours do not exceed 14 consecutive hours before mandatory 10-hour rest (HOS)

Here $x_{ijk} = 1$ if truck $k$ travels arc $(i,j)$, $y_{ik} = 1$ if order $i$ is assigned to truck $k$, $c_{ij}$ is the cost of traversing arc $(i,j)$, $q_i$ is order weight, and $Q_k$ is the capacity of truck $k$.

**Why is this hard?** The CVRPTW is NP-hard. With 300 orders and 15 trucks, the number of feasible assignments exceeds $10^{353}$ -- a figure that dwarfs the estimated number of atoms in the observable universe ($\approx 10^{80}$). No exact algorithm can solve this in practical time. We need heuristics.

To put it in concrete terms: imagine you have 15 delivery vans, each with a weight limit, serving 300 customers across the country. Each driver works a maximum of 14 hours before taking a mandatory 10-hour break. You need the cheapest set of routes where every order gets delivered, no truck is overloaded, and no driver violates rest regulations. Now imagine you need to solve this every day as orders change. That is what our algorithms do.

### Key Results

| KPI | Value |
|:---|---:|
| Total fleet cost | $126,854 |
| Routes dispatched | 33 |
| Orders fulfilled | 300 / 300 (100%) |
| Total distance | 87,391 mi |
| CO2 emitted | 162 tonnes |
| Average cost per mile | $1.45 |
| Average truck utilisation | 96.4% |



---
## Solver Strategy

Routing 300 orders across 15 trucks is a combinatorial problem. The solution space grows factorially with the number of orders -- there is no shortcut, no closed-form answer. We built four solvers of increasing sophistication and ran them head-to-head.

| Solver | Approach | Runtime | When to use it |
|:---|:---|:---|:---|
| **Greedy** | Nearest-insertion heuristic | ~0.03s | Real-time fallback when a truck breaks down mid-day |
| **Column Generation** | LP relaxation with pricing subproblem | ~42s | When you need a provable lower bound on cost |
| **ALNS** | Adaptive metaheuristic (destroy/repair) | ~10 min | Daily batch dispatch -- best cost in our experiments |
| **Pareto** | Multi-objective (cost vs CO2 vs time) | Variable | Monthly strategic review of fleet trade-offs |

### How ALNS Works

The Adaptive Large Neighbourhood Search (Ropke and Pisinger, 2006) operates in a loop of three steps, repeated 6,000 times:

**Step 1 -- Destroy.** Remove a fraction of orders from the current solution. The fraction $q$ is drawn uniformly from $[0.10, 0.35]$. Four destroy operators compete for selection:

- Random removal -- pulls orders at random for diversification
- Worst removal -- targets the most expensive orders
- Shaw removal -- removes geographically clustered orders to enable structural changes
- Route removal -- deletes an entire underperforming route

**Step 2 -- Repair.** Reinsert removed orders into feasible positions:

- Greedy insertion -- cheapest available slot
- Regret-2 insertion -- prioritises orders where the gap between the best and second-best slot is largest (the ones that would hurt most to defer)
- Perturbed insertion -- greedy plus random noise to encourage exploration

**Step 3 -- Accept or reject.** The new solution is evaluated using a Simulated Annealing criterion:

$$P(\text{accept}) = \begin{cases} 1 & \text{if } \Delta C \leq 0 \\ e^{-\Delta C / T} & \text{if } \Delta C > 0 \end{cases}$$

where $\Delta C = C_{\text{new}} - C_{\text{current}}$ is the cost difference, and $T$ is the temperature, which cools geometrically each iteration: $T_{t+1} = 0.9997 \cdot T_t$, starting from $T_0 = 500$.

In practical terms: at the start, the algorithm is willing to accept worse solutions roughly half the time, which lets it escape local optima and explore broadly. As the temperature drops toward zero, it becomes increasingly selective and converges toward the best solution found. This tension between exploration and exploitation is what makes metaheuristics work on hard problems.

**Operator adaptation.** Every 100 iterations, the algorithm updates operator weights based on recent performance:

$$w_i^{(s+1)} = (1 - \alpha) \cdot w_i^{(s)} + \alpha \cdot \frac{\pi_i}{\theta_i}$$

Operators earn scores: 33 points for finding a new global best, 9 for improving the current solution, 1 for an accepted but non-improving move. Over 6,000 iterations, the algorithm discovers that route-destroy paired with regret-2 repair is the most productive combination for this problem instance.

### Why Monte Carlo on Top?

A deterministic solution assumes perfect information: exact fuel prices, precise travel times, no mechanical failures. None of that holds in practice.

We stress-test every solver's output with 1,000 Monte Carlo trials, applying three sources of uncertainty drawn from industry-calibrated distributions:

| Factor | Distribution | Parameters | Source |
|:---|:---|:---|:---|
| Travel time | Log-normal | sigma = 20% | Figliozzi (2010), inter-city trucking CoV |
| Fuel price | Normal | sigma = 15% around $3.75/gal | EIA Weekly Petroleum Status Report |
| Truck breakdown | Bernoulli | p = 0.05 per route | FMCSA roadside inspection out-of-service rate |

With 1,000 independent trials, the Central Limit Theorem gives a standard error of $\sigma / \sqrt{1000}$, yielding roughly 3% precision on cost estimates at the 95% confidence level. That is more than sufficient for operational decision-making.

### Setup -- Corporate Visual Identity

All charts use a custom Plotly template designed around the corporate brand colours: navy (#00263A), harvest gold (#D4A843), and field green (#4A7C59) on a cream (#FAF7F2) background, with Segoe UI typography.


In [3]:

# -- imports & data --
import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# -- file paths --
RESULTS = Path("results")
fleet   = pd.read_csv(RESULTS / "fleet_summary.csv")
routes  = pd.read_csv(RESULTS / "route_details.csv")
orders  = pd.read_csv(RESULTS / "order_assignments.csv")
timeline = pd.read_csv(RESULTS / "dispatch_timeline.csv")

with open(RESULTS / "solution_metadata.json") as f:
    metadata = json.load(f)

# -- ADM brand palette (Archer Daniels Midland) --
ADM = {
    "navy":       "#00263A",   # primary corporate
    "gold":       "#D4A843",   # harvest gold
    "green":      "#4A7C59",   # field green
    "crimson":    "#C8102E",   # accent / alert
    "sky":        "#5B9BD5",   # secondary blue
    "wheat":      "#E8D5B7",   # warm neutral
    "slate":      "#6B7B8D",   # cool grey
    "earth":      "#8B6914",   # dark gold
    "cream":      "#FAF7F2",   # background
    "light_grey": "#E5E5E5",   # grid lines
    "dark_text":  "#2D2D2D",   # body text
    "teal":       "#2D8E8E",   # sustainability accent
}

# ordered sequence for multi-series charts
ADM_SEQ = [ADM["navy"], ADM["gold"], ADM["green"], ADM["crimson"],
           ADM["sky"], ADM["earth"], ADM["slate"], ADM["teal"]]

# -- Utility: hex to rgba string --
def _hex_rgba(hex_color, alpha=0.35):
    """Convert hex colour to rgba string with given opacity."""
    h = hex_color.lstrip("#")
    r, g, b = int(h[:2], 16), int(h[2:4], 16), int(h[4:6], 16)
    return f"rgba({r},{g},{b},{alpha})"

# -- Plotly template --
adm_template = go.layout.Template()
adm_template.layout = go.Layout(
    font=dict(family="Segoe UI, Helvetica Neue, Arial", size=13, color=ADM["dark_text"]),
    title=dict(font=dict(size=20, color=ADM["navy"]), x=0.01, xanchor="left"),
    paper_bgcolor=ADM["cream"],
    plot_bgcolor=ADM["cream"],
    colorway=ADM_SEQ,
    xaxis=dict(showgrid=False, linecolor=ADM["navy"], linewidth=1.5,
               ticks="outside", tickcolor=ADM["navy"], zeroline=False),
    yaxis=dict(showgrid=True, gridcolor=ADM["light_grey"], gridwidth=0.5,
               linecolor=ADM["navy"], linewidth=0, ticks="", zeroline=False),
    margin=dict(l=60, r=30, t=105, b=50),
    hoverlabel=dict(bgcolor="white", font_size=12, bordercolor=ADM["navy"]),
    legend=dict(orientation="h", yanchor="bottom", y=1.10, xanchor="left", x=0,
                bgcolor="rgba(0,0,0,0)", font=dict(size=11)),
    bargap=0.25,
)
pio.templates["adm_template"] = adm_template
pio.templates.default = "adm_template"

print(f"Loaded {len(routes)} routes, {len(orders)} orders, {len(timeline)} timeline events")
print(f"ADM template registered · _hex_rgba helper available")


Loaded 33 routes, 300 orders, 245 timeline events
ADM template registered · _hex_rgba helper available



---
## 1 - Cost Architecture

The fleet spent $126,854 across 33 routes. That total breaks down into three components:

$$\text{Total Cost} = \underbrace{\text{Fuel}}_{47\%} + \underbrace{\text{Labour}}_{43\%} + \underbrace{\text{Maintenance}}_{10\%}$$

Fuel dominates because diesel at $3.75 per gallon compounds over long distances. Labour runs a close second -- not because drivers are overpaid, but because HOS regulations force 10-hour rest breaks that keep drivers on the payroll without moving freight.

This split tells us where to focus. Fuel savings come from shorter routes (better optimisation). Labour savings come from tighter scheduling that avoids triggering rest breaks.


In [4]:
# ---
# CHART 1 ·  Fleet Cost Breakdown -- Horizontal Stacked Bar
# ---

cost_data = pd.DataFrame({
    "Component": ["Fuel", "Labour", "Maintenance"],
    "Cost": [59584.66, 54160.39, 13108.64],
    "Colour": [ADM["navy"], ADM["gold"], ADM["green"]]
})
cost_data["Pct"] = (cost_data["Cost"] / cost_data["Cost"].sum() * 100).round(1)
cost_data["Label"] = cost_data.apply(
    lambda r: f"${r['Cost']:,.0f}  ({r['Pct']:.0f}%)", axis=1
)

fig = go.Figure()
for _, row in cost_data.iterrows():
    fig.add_trace(go.Bar(
        x=[row["Cost"]], y=["Fleet Total"],
        orientation="h", name=row["Component"],
        marker_color=row["Colour"],
        text=row["Label"], textposition="inside",
        textfont=dict(color="white", size=14, family="Segoe UI Semibold"),
        hovertemplate=f"<b>{row['Component']}</b><br>${row['Cost']:,.0f} ({row['Pct']}%)<extra></extra>",
    ))

fig.update_layout(
    title="<b>Fleet Cost Architecture</b><br><sup>Where every dollar goes -- $126,854 total operating cost</sup>",
    barmode="stack",
    xaxis=dict(title="Cost ($)", tickprefix="$", tickformat=",.0f"),
    yaxis=dict(visible=False),
    height=220, width=900,
    showlegend=True,
)
fig.show()


---
## 2 - Route Cost Distribution

Not all routes are equal. The cheapest route costs $581 (a short hop to Santa Claus, IN), while the most expensive runs $8,796 (a cross-country haul through Las Vegas, Portland, and Forks, WA).

Each route below is coloured by its dominant cost driver -- fuel or labour. Routes above the $5,000 line account for a disproportionate share of total spend and are the first candidates for consolidation or mode-shifting to rail.


In [5]:
# ---
# CHART 2 ·  Route Cost Lollipop Chart (sorted, with threshold)
# ---

df = routes.sort_values("Total$", ascending=True).reset_index(drop=True)
df["route_label"] = "R" + df["Route"].astype(str)

# colour by whether fuel or labour dominates
df["dominant"] = np.where(df["Fuel$"] > df["Labour$"], "Fuel-dominated", "Labour-dominated")
colour_map = {"Fuel-dominated": ADM["navy"], "Labour-dominated": ADM["gold"]}

fig = go.Figure()

# stems
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(
        x=[0, row["Total$"]], y=[row["route_label"], row["route_label"]],
        mode="lines", line=dict(color=ADM["light_grey"], width=1.5),
        showlegend=False, hoverinfo="skip",
    ))

# dots
for dom, grp in df.groupby("dominant"):
    fig.add_trace(go.Scatter(
        x=grp["Total$"], y=grp["route_label"],
        mode="markers", name=dom,
        marker=dict(size=9, color=colour_map[dom], line=dict(width=1, color="white")),
        hovertemplate="<b>Route %{y}</b><br>$%{x:,.0f}<extra></extra>",
    ))

# threshold line
fig.add_vline(x=5000, line_dash="dash", line_color=ADM["crimson"], line_width=1.5,
              annotation_text="$5k threshold", annotation_position="top right",
              annotation_font_color=ADM["crimson"])

fig.update_layout(
    title="<b>Route Cost Distribution</b><br><sup>Each dot = one route · Colour = primary cost driver</sup>",
    xaxis=dict(title="Total Route Cost ($)", tickprefix="$", tickformat=",.0f"),
    yaxis=dict(title="", tickfont=dict(size=9)),
    height=750, width=900,
    legend=dict(orientation="h", yanchor="top", y=-0.05, xanchor="left", x=0),
)
fig.show()


---
## 3 - Driver Workload Balance

Five drivers share 33 routes. If one driver is overloaded, HOS violations become likely. If another is underutilised, we are paying for idle capacity. The three panels below show how routes, miles, and cost responsibility distribute across the team.


In [6]:
# ---
# CHART 3 · Driver Workload -- Grouped Bar Chart
# ---

driver_stats = routes.groupby("Driver").agg(
    routes=("Route", "count"),
    distance=("Dist_mi", "sum"),
    cost=("Total$", "sum"),
    weight=("Weight_kg", "sum"),
    hours=("Work_hrs", "sum"),
).reset_index().sort_values("cost", ascending=False)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Routes Assigned", "Distance (miles)", "Cost Responsibility ($)"),
    shared_yaxes=True, horizontal_spacing=0.08,
)

fig.add_trace(go.Bar(
    x=driver_stats["routes"], y=driver_stats["Driver"],
    orientation="h", marker_color=ADM["navy"],
    text=driver_stats["routes"], textposition="outside",
    hovertemplate="<b>%{y}</b><br>%{x} routes<extra></extra>",
    showlegend=False,
), row=1, col=1)

fig.add_trace(go.Bar(
    x=driver_stats["distance"], y=driver_stats["Driver"],
    orientation="h", marker_color=ADM["gold"],
    text=driver_stats["distance"].apply(lambda v: f"{v:,.0f}"), textposition="outside",
    hovertemplate="<b>%{y}</b><br>%{x:,.0f} mi<extra></extra>",
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Bar(
    x=driver_stats["cost"], y=driver_stats["Driver"],
    orientation="h", marker_color=ADM["green"],
    text=driver_stats["cost"].apply(lambda v: f"${v:,.0f}"), textposition="outside",
    hovertemplate="<b>%{y}</b><br>$%{x:,.0f}<extra></extra>",
    showlegend=False,
), row=1, col=3)

fig.update_layout(
    title="<b>Driver Workload Balance</b><br><sup>Routes, distance, and cost responsibility per driver</sup>",
    height=350, width=1000,
)
fig.update_xaxes(showgrid=False)
fig.show()


---
## 4 - Truck Class Efficiency

The Company's fleet spans four FHWA vehicle classes. Larger trucks carry more cargo but burn more diesel per mile. To compare apples to apples, we normalise by both distance and payload:

$$\text{Cost per ton-mile} = \frac{\text{Total Route Cost}}{\text{Payload (tonnes)} \times \text{Distance (mi)}}$$

A lower value means the truck moves more freight per dollar. Class 8 trucks tend to win on economies of scale, but the picture is more nuanced than that -- route length and stop count matter too.


In [7]:
# ---
# CHART 4 · Cost per Ton-Mile by Truck Class -- Violin + Strip
# ---

truck_class_map = {
    "000001": "Class 8", "000002": "Class 8",
    "000003": "Class 8", "000004": "Class 8",
    "000005": "Class 8",
}

routes["ton_miles"] = (routes["Weight_kg"] / 1000) * routes["Dist_mi"]
routes["cost_per_ton_mi"] = routes["Total$"] / routes["ton_miles"]

driver_truck = {
    "Steve": "Class 8 (16.06 m)",
    "Dan":   "Class 8 (16.06 m)",
    "Joe":   "Class 7 (14.82 m)",
    "William": "Class 6 (13.98 m)",
    "Tom":   "Class 5 (11.57 m)",
}
routes["Truck_Class"] = routes["Driver"].map(driver_truck)

class_colours = {
    "Class 8 (16.06 m)": ADM["navy"],
    "Class 7 (14.82 m)": ADM["gold"],
    "Class 6 (13.98 m)": ADM["green"],
    "Class 5 (11.57 m)": ADM["crimson"],
}

class_capacity = {
    "Class 8 (16.06 m)": 10000,
    "Class 7 (14.82 m)": 7500,
    "Class 6 (13.98 m)": 6000,
    "Class 5 (11.57 m)": 4000,
}

fig = go.Figure()
for cls in ["Class 8 (16.06 m)", "Class 7 (14.82 m)", "Class 6 (13.98 m)", "Class 5 (11.57 m)"]:
    mask = routes["Truck_Class"] == cls
    sub = routes[mask]
    n = mask.sum()
    med = sub["cost_per_ton_mi"].median()

    # violin for distribution shape
    fig.add_trace(go.Violin(
        y=sub["cost_per_ton_mi"],
        name=f"{cls}  (n={n})",
        marker_color=class_colours[cls],
        fillcolor=class_colours[cls],
        opacity=0.25,
        line_color=class_colours[cls],
        meanline_visible=True,
        box_visible=True,
        points="all",
        pointpos=0,
        jitter=0.35,
        marker=dict(size=8, opacity=0.8, line=dict(width=1, color="white")),
        hovertemplate="<b>%{text}</b><br>$%{y:.4f}/ton-mi<extra></extra>",
        text=["R" + str(r) for r in sub["Route"]],
        showlegend=True,
    ))

fig.update_layout(
    title="<b>Cost Efficiency by Truck Class</b><br>"
          "<sup>Cost per ton-mile (lower = better) -- violin shows distribution, dots = individual routes</sup>",
    yaxis=dict(title="Cost per Ton-Mile ($)", tickprefix="$", tickformat=".3f"),
    xaxis=dict(title=""),
    height=500, width=900,
    legend=dict(orientation="h", y=-0.15, x=0.5, xanchor="center"),
    violingap=0.3,
    violinmode="overlay",
)
fig.show()


---
## 5 - Carbon Footprint Analysis

Every gallon of diesel burned produces 10.18 kg of CO2 (EPA emission factor for highway diesel). The fleet's total emissions follow directly from fuel consumption:

$$\text{CO}_2 = \sum_{r \in \text{routes}} \frac{d_r}{\text{MPG}_r} \times 10.18 \;\text{kg}$$

where $d_r$ is the route distance in miles and $\text{MPG}_r$ is the fuel economy of the assigned truck.

The bubble chart below plots distance against CO2 for each route, with bubble size proportional to payload weight. The relationship is almost perfectly linear: long, heavy routes are the biggest emitters, and they are also the best candidates for electrification or rail intermodal on the trunk segment.


In [8]:
# ---
# CHART 5 · Carbon Bubble Chart -- Distance vs CO2 vs Payload
# ---

fig = go.Figure()

for cls in ["Class 8 (16.06 m)", "Class 7 (14.82 m)", "Class 6 (13.98 m)", "Class 5 (11.57 m)"]:
    mask = routes["Truck_Class"] == cls
    sub = routes[mask]
    fig.add_trace(go.Scatter(
        x=sub["Dist_mi"], y=sub["CO2_kg"],
        mode="markers", name=cls,
        marker=dict(
            size=sub["Weight_kg"] / 600,  # scale for readability
            color=class_colours[cls],
            opacity=0.75,
            line=dict(width=1, color="white"),
        ),
        hovertemplate=(
            "<b>Route %{customdata[0]}</b> (%{customdata[1]})<br>"
            "Distance: %{x:,.0f} mi<br>"
            "CO2: %{y:,.0f} kg<br>"
            "Payload: %{customdata[2]:,.0f} kg"
            "<extra></extra>"
        ),
        customdata=sub[["Route", "Driver", "Weight_kg"]].values,
    ))

# trend line
z = np.polyfit(routes["Dist_mi"], routes["CO2_kg"], 1)
x_line = np.linspace(routes["Dist_mi"].min(), routes["Dist_mi"].max(), 100)
fig.add_trace(go.Scatter(
    x=x_line, y=np.polyval(z, x_line),
    mode="lines", name="Trend",
    line=dict(color=ADM["crimson"], width=2, dash="dash"),
    hoverinfo="skip",
))

fig.update_layout(
    title="<b>Carbon Footprint by Route</b><br><sup>Bubble size = payload weight · Dashed line = linear trend</sup>",
    xaxis=dict(title="Route Distance (miles)"),
    yaxis=dict(title="CO2 Emissions (kg)"),
    height=550, width=900,
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="left", x=0),
)
fig.show()


---
## 6 - Capacity Utilisation

How full is each truck? This is a direct measure of packing efficiency. The ALNS solver's regret-2 repair operator is designed to fill trucks as close to capacity as possible -- leaving a nearly empty truck on the road is pure waste. The target is above 95%.


In [9]:
# ---
# CHART 6 · Capacity Utilisation -- Sorted Horizontal Bar
# ---

routes_sorted = routes.sort_values("Weight_kg", ascending=True).copy()
routes_sorted["util_pct"] = routes_sorted["Weight_kg"] / 10000 * 100
routes_sorted["route_label"] = "R" + routes_sorted["Route"].astype(str) + "  (" + routes_sorted["Driver"] + ")"

colours = [ADM["crimson"] if u < 90 else ADM["gold"] if u < 96 else ADM["navy"]
           for u in routes_sorted["util_pct"]]

fig = go.Figure()
fig.add_trace(go.Bar(
    y=routes_sorted["route_label"],
    x=routes_sorted["util_pct"],
    orientation="h",
    marker_color=colours,
    text=routes_sorted["util_pct"].apply(lambda v: f"{v:.0f}%"),
    textposition="outside", textfont=dict(size=10),
    hovertemplate="<b>%{y}</b><br>Utilisation: %{x:.1f}%<br>Payload: %{customdata:,.0f} kg<extra></extra>",
    customdata=routes_sorted["Weight_kg"],
))

# reference lines
fig.add_vline(x=100, line_dash="solid", line_color=ADM["navy"], line_width=2,
              annotation_text="Truck capacity (10 t)", annotation_position="top right",
              annotation_font=dict(color=ADM["navy"], size=11))
fig.add_vline(x=95, line_dash="dot", line_color=ADM["gold"], line_width=1,
              annotation_text="95% target", annotation_position="bottom left",
              annotation_font=dict(color=ADM["earth"], size=10))

# colour legend annotation
fig.add_annotation(
    x=0.98, y=-0.06, xref="paper", yref="paper", showarrow=False,
    text="<span style='color:#00263A'>>=96%</span>  |  "
         "<span style='color:#D4A843'>90-95%</span>  |  "
         "<span style='color:#C8102E'><90%</span>",
    font=dict(size=11), xanchor="right",
)

# summary stats
pct = routes_sorted["util_pct"].median()
n_high = (routes_sorted["util_pct"] >= 96).sum()
fig.add_annotation(
    x=0.02, y=-0.06, xref="paper", yref="paper", showarrow=False,
    text=f"Median utilisation: <b>{pct:.0f}%</b>  |  {n_high} of {len(routes_sorted)} routes at >=96%",
    font=dict(size=11, color=ADM["navy"]), xanchor="left",
)

fig.update_layout(
    title="<b>Truck Capacity Utilisation</b><br>"
          "<sup>Payload as % of 10-tonne nominal capacity -- sorted lowest to highest</sup>",
    xaxis=dict(title="Utilisation (%)", range=[78, 108]),
    yaxis=dict(title="", tickfont=dict(size=10)),
    height=max(500, len(routes_sorted) * 22),
    width=900,
    showlegend=False,
    margin=dict(l=160, r=50, t=105, b=70),
)
fig.show()


---
## 7 - Geographic Reach

All 33 routes radiate from Cincinnati, OH. The map reveals why cost varies so dramatically: a run to Portland, OR covers 4,400+ miles round-trip, while Columbus, OH is barely 200 miles away. West Coast routes dominate both cost and distance -- and they are also the routes most likely to trigger multiple HOS rest breaks.


In [10]:
# ---
# CHART 7 · Route Network Map -- Plotly Scattergeo
# ---

# city coords from config
COORDS = {
    "Cincinnati, OH": (39.1031, -84.5120),
    "Albany, NY": (42.6526, -73.7562), "Baton Rouge, LA": (30.4515, -91.1871),
    "Boston, MA": (42.3601, -71.0589), "Chicago, IL": (41.8781, -87.6298),
    "Colorado Springs, CO": (38.8339, -104.8214), "Newport, KY": (39.0914, -84.4958),
    "Oshkosh, WI": (44.0247, -88.5426), "Philadelphia, PA": (39.9526, -75.1652),
    "Portland, MA": (43.6591, -70.2568), "Portland, OR": (45.5152, -122.6784),
    "Santa Claus, IN": (38.1200, -86.9141), "Washington, DC": (38.9072, -77.0369),
    "Cambridge, MA": (42.3736, -71.1097), "Charlotte, NC": (35.2271, -80.8431),
    "Decatur, IL": (39.8403, -88.9548), "Lexington, SC": (33.9815, -81.2368),
    "Lincoln, NE": (40.8136, -96.7026), "Los Angeles, CA": (34.0522, -118.2437),
    "Orlando, FL": (28.5383, -81.3792), "Davis, CA": (38.5449, -121.7405),
    "Detroit, MI": (42.3314, -83.0458), "Hilton Head, SC": (32.2163, -80.7526),
    "Las Vegas, NV": (36.1699, -115.1398), "Lexington, KY": (38.0406, -84.5037),
    "Memphis TN": (35.1495, -90.0490), "Tampa, FL": (27.9506, -82.4572),
    "Toronto, Canada": (43.6532, -79.3832), "Albany, ID": (42.0966, -114.2598),
    "Columbus, OH": (39.9612, -82.9988), "Forks, WA": (47.9504, -124.3855),
    "Miami, FL": (25.7617, -80.1918), "San Francisco, CA": (37.7749, -122.4194),
    "Seattle, WA": (47.6062, -122.3321), "Salt Lake City, UT": (40.7608, -111.8910),
    "Dallas, TX": (32.7767, -96.7970), "Grand Rapids, MI": (42.9634, -85.6681),
    "Key West, FL": (24.5551, -81.7800), "Nashville, TN": (36.1627, -86.7816),
    "Omaha, NE": (41.2565, -95.9345), "Salem, MA": (42.5195, -70.8967),
    "Birmingham, AL": (33.5186, -86.8104), "Denver, CO": (39.7392, -104.9903),
    "St Louis, MO": (38.6270, -90.1994), "Buffalo, NY": (42.8864, -78.8784),
    "New York City, NY": (40.7128, -74.0060), "Louisville, KY": (38.2527, -85.7585),
    "Indianapolis, IN": (39.7684, -86.1581), "Spokane, WA": (47.6588, -117.4260),
    "Frankfort, KY": (38.2009, -84.8733), "Pittsburg, PA": (40.4406, -79.9959),
}

DEPOT = COORDS["Cincinnati, OH"]

fig = go.Figure()

# draw route arcs
for _, row in routes.iterrows():
    cities_str = row["Cities"]
    city_list = [c.strip() for c in cities_str.split("->")]
    for i in range(len(city_list) - 1):
        c1, c2 = city_list[i], city_list[i+1]
        if c1 in COORDS and c2 in COORDS:
            lat1, lon1 = COORDS[c1]
            lat2, lon2 = COORDS[c2]
            fig.add_trace(go.Scattergeo(
                lon=[lon1, lon2], lat=[lat1, lat2],
                mode="lines",
                line=dict(width=1.2, color=ADM["navy"]),
                opacity=0.35,
                showlegend=False, hoverinfo="skip",
            ))

# destination markers
dest_orders = orders.groupby("Destination").agg(
    n_orders=("Order", "count"), total_wt=("Weight_kg", "sum")
).reset_index()
dest_orders["lat"] = dest_orders["Destination"].map(lambda c: COORDS.get(c, (None, None))[0])
dest_orders["lon"] = dest_orders["Destination"].map(lambda c: COORDS.get(c, (None, None))[1])
dest_orders = dest_orders.dropna(subset=["lat", "lon"])

fig.add_trace(go.Scattergeo(
    lon=dest_orders["lon"], lat=dest_orders["lat"],
    mode="markers",
    marker=dict(
        size=dest_orders["n_orders"] * 1.5 + 4,
        color=ADM["gold"], opacity=0.85,
        line=dict(width=1, color=ADM["navy"]),
    ),
    text=dest_orders.apply(lambda r: f"{r['Destination']}<br>{r['n_orders']} orders", axis=1),
    hovertemplate="%{text}<extra></extra>",
    name="Destinations",
))

# depot star
fig.add_trace(go.Scattergeo(
    lon=[DEPOT[1]], lat=[DEPOT[0]],
    mode="markers+text",
    marker=dict(size=18, color=ADM["sky"], symbol="star",
                line=dict(width=1.5, color="white")),
    text=["DEPOT"], textposition="top center",
    textfont=dict(color=ADM["crimson"], size=11, family="Segoe UI Semibold"),
    name="Cincinnati Depot",
    hovertemplate="<b>Cincinnati, OH</b><br>Warehouse & Depot<extra></extra>",
))

fig.update_geos(
    scope="north america",
    showland=True, landcolor="#F0EDE6",
    showlakes=True, lakecolor="#D6E8F0",
    showcountries=True, countrycolor=ADM["light_grey"],
    showsubunits=True, subunitcolor=ADM["light_grey"],
    bgcolor=ADM["cream"],
    lonaxis=dict(range=[-130, -65]),
    lataxis=dict(range=[22, 52]),
)

fig.update_layout(
    title="<b>Route Network Map</b><br><sup>33 routes from Cincinnati depot · Bubble size = order count per destination</sup>",
    height=600, width=1000,
    legend=dict(x=0.01, y=0.01),
)
fig.show()

In [11]:
ADM

{'navy': '#00263A',
 'gold': '#D4A843',
 'green': '#4A7C59',
 'crimson': '#C8102E',
 'sky': '#5B9BD5',
 'wheat': '#E8D5B7',
 'slate': '#6B7B8D',
 'earth': '#8B6914',
 'cream': '#FAF7F2',
 'light_grey': '#E5E5E5',
 'dark_text': '#2D2D2D',
 'teal': '#2D8E8E'}


---
## 8 - ALNS Operator Adaptation

The ALNS solver does not treat its operators equally. After every 100 iterations, it updates their selection probabilities based on how well each has performed:

$$w_i^{(s+1)} = (1 - \alpha) \cdot w_i^{(s)} + \alpha \cdot \frac{\pi_i}{\theta_i}$$

where $\pi_i$ is the accumulated score for operator $i$ during the segment, $\theta_i$ is the number of times it was invoked, and $\alpha$ is a reactivity parameter.

After 6,000 iterations, the final weights tell us which operators the algorithm found most productive for this problem:

- **Route-destroy** (52%) -- removing entire underperforming routes proved the most impactful structural move.
- **Regret-2 repair** (54%) -- reinserting orders where delaying them would cost the most, rather than just taking the cheapest slot.


In [12]:
# ---
# CHART 8 · ALNS Operator Weights -- Radial / Polar Bar
# ---

destroy_w = metadata["operator_weights"]["destroy"]
repair_w  = metadata["operator_weights"]["repair"]

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "polar"}, {"type": "polar"}]],
    subplot_titles=("Destroy Operators", "Repair Operators"),
)

# Destroy
d_names = list(destroy_w.keys())
d_vals = [v * 100 for v in destroy_w.values()]
d_colours = [ADM["navy"], ADM["gold"], ADM["green"], ADM["crimson"]]

fig.add_trace(go.Barpolar(
    r=d_vals, theta=d_names,
    marker=dict(color=d_colours, line=dict(color="white", width=2)),
    text=[f"{v:.0f}%" for v in d_vals],
    hovertemplate="<b>%{theta}</b><br>Weight: %{r:.1f}%<extra></extra>",
), row=1, col=1)

# Repair
r_names = list(repair_w.keys())
r_vals = [v * 100 for v in repair_w.values()]
r_colours = [ADM["sky"], ADM["earth"], ADM["teal"]]

fig.add_trace(go.Barpolar(
    r=r_vals, theta=r_names,
    marker=dict(color=r_colours, line=dict(color="white", width=2)),
    text=[f"{v:.0f}%" for v in r_vals],
    hovertemplate="<b>%{theta}</b><br>Weight: %{r:.1f}%<extra></extra>",
), row=1, col=2)

fig.update_layout(
    title="<b>ALNS Operator Adaptation</b><br><sup>Final selection weights after 6,000 iterations · Higher = more effective</sup>",
    height=450, width=900,
    polar=dict(bgcolor=ADM["cream"], radialaxis=dict(visible=True, ticksuffix="%")),
    polar2=dict(bgcolor=ADM["cream"], radialaxis=dict(visible=True, ticksuffix="%")),
    showlegend=False,
)
fig.show()


---
## 9 - Dispatch Timeline (Gantt)

This is where the optimisation hits the road. The Gantt chart translates abstract route assignments into a minute-by-minute execution plan for every driver, truck, and route.

### HOS Scheduling Constraints (FMCSA)

| Rule | Constraint | Effect on Cost |
|:---|:---|:---|
| 14-hour work window | Max 14 consecutive hours on duty | Caps route duration per shift |
| 10-hour mandatory rest | Must rest 10h before next shift | Adds "dead time" that inflates labour cost |
| 30-min loading | Warehouse preparation at departure | Fixed overhead per route |
| 35-min stop overhead | Unload plus paperwork per delivery | Scales with number of stops |

Each row in the chart is one route assignment (Driver / Truck / Route). Bars are colour-coded: navy for loading at the warehouse, green for delivery stops, blue for the return leg, and red cross-hatching for mandatory HOS rest periods.

Wide red bars mean the route spans multiple days because a rest break was triggered. These are the most expensive routes in the fleet -- the driver is accumulating wages without moving freight.

The core scheduling insight: a route that triggers even one rest break roughly doubles its wall-clock duration. The ALNS solver minimises rest breaks by packing deliveries tightly within each 14-hour window.


In [13]:

# ---
# CHART 9 · Dispatch Gantt Chart (Dash-style)
# ---

gantt_df = timeline.copy()
gantt_df["Arrive"] = pd.to_datetime(gantt_df["Arrive"])
gantt_df["Depart"] = pd.to_datetime(gantt_df["Depart"])

# Activity styling -- matching the Dash dashboard design
act_cfg = {
    "loading":        {"color": ADM["navy"],    "symbol": "[L]", "label": "Loading"},
    "delivery":       {"color": ADM["green"],   "symbol": "[D]", "label": "Delivery"},
    "return":         {"color": ADM["sky"],     "symbol": "[R]", "label": "Return"},
    "mandatory_rest": {"color": ADM["crimson"], "symbol": "[H]", "label": "HOS Rest"},
}

# Build descriptive y-axis labels: "Driver  •  T{truck}  •  R{route}"
gantt_df["drv_label"] = (
    gantt_df["Driver"] + " | T" +
    gantt_df["Truck"].astype(str) + " | R" +
    gantt_df["Route"].astype(str)
)

# Sort rows by earliest departure for visual consistency
first_depart = gantt_df.groupby("drv_label")["Arrive"].min()
sorted_labels = first_depart.sort_values().index.tolist()

fig = go.Figure()

# Alternating row shading for readability
for idx, lbl in enumerate(sorted_labels):
    if idx % 2 == 0:
        fig.add_hrect(
            y0=idx - 0.42, y1=idx + 0.42,
            fillcolor=_hex_rgba(ADM["navy"], 0.025),
            line_width=0, layer="below",
        )

# Plot activity bars -- one trace per activity type per y-label group
for activity, cfg in act_cfg.items():
    sub = gantt_df[gantt_df["Activity"] == activity].copy()
    if sub.empty:
        continue
    dur_ms = (sub["Depart"] - sub["Arrive"]).dt.total_seconds() * 1000

    # Build rich per-event hover text
    hover = []
    for _, row in sub.iterrows():
        dh = (row["Depart"] - row["Arrive"]).total_seconds() / 3600
        hover.append(
            f"<b>{cfg['symbol']} {row['City']}</b><br>"
            f"<span style='color:{cfg['color']}'>--</span> {cfg['label']}<br>"
            f"{row['Arrive'].strftime('%b %d, %H:%M')} -> "
            f"{row['Depart'].strftime('%H:%M')}<br>"
            f"Duration: <b>{dh:.1f} h</b><br>"
            f"Driver: {row['Driver']} | Truck {row['Truck']}"
        )

    # Rest breaks get cross-hatching pattern
    if activity == "mandatory_rest":
        mkr = dict(
            color=_hex_rgba(ADM["crimson"], 0.25),
            line=dict(width=1.5, color=ADM["crimson"]),
            cornerradius=4,
            pattern=dict(shape="/",
                         fgcolor=_hex_rgba(ADM["crimson"], 0.45),
                         size=6, solidity=0.4),
        )
    else:
        mkr = dict(
            color=_hex_rgba(cfg["color"], 0.78),
            line=dict(width=1.2, color=cfg["color"]),
            cornerradius=4,
        )

    fig.add_trace(go.Bar(
        y=sub["drv_label"],
        x=dur_ms,
        base=sub["Arrive"].dt.strftime("%Y-%m-%dT%H:%M:%S"),
        orientation="h",
        marker=mkr,
        width=0.55,
        showlegend=False,
        customdata=hover,
        hovertemplate="%{customdata}<extra></extra>",
    ))

# Invisible legend traces for a clean, uniform legend
for act, cfg in act_cfg.items():
    mkr = dict(color=cfg["color"], cornerradius=3)
    if act == "mandatory_rest":
        mkr["pattern"] = dict(shape="/",
                              fgcolor=_hex_rgba(ADM["crimson"], 0.6))
    fig.add_trace(go.Bar(
        y=[None], x=[None], marker=mkr,
        name=f"{cfg['symbol']} {cfg['label']}",
        showlegend=True,
    ))

n_rows = len(sorted_labels)
fig.update_layout(
    title="<b>Fleet Dispatch Timeline</b><br>"
          "<sup>HOS-compliant schedule -- Feb 3 to Mar 1, 2026</sup>",
    barmode="stack",
    height=max(480, n_rows * 28 + 120),
    width=1100,
    xaxis=dict(
        type="date", title="",
        gridcolor=_hex_rgba(ADM["navy"], 0.06),
        gridwidth=0.5,
        dtick=86_400_000,
        tickformat="%b %d\n%a",
        tickfont=dict(size=10, color=ADM["slate"]),
        side="top",
        showline=True, linecolor=ADM["light_grey"], linewidth=1,
    ),
    yaxis=dict(
        title="", autorange="reversed",
        tickfont=dict(size=9, color=ADM["navy"]),
        categoryorder="array",
        categoryarray=sorted_labels,
        gridcolor="rgba(0,0,0,0)",
    ),
    legend=dict(
        orientation="h", y=-0.06, x=0.5, xanchor="center",
        bgcolor="rgba(250,247,242,0.9)",
        bordercolor=ADM["light_grey"], borderwidth=1,
        font=dict(size=10.5, color=ADM["navy"]),
        itemwidth=30,
    ),
    bargap=0.08,
)
fig.show()




---
## 10 - Cost vs Distance Efficiency

Longer routes should be cheaper per mile because fixed costs (loading, overhead) get spread across more distance. The scatter below tests whether that holds up in our data, and fits an amortisation curve to quantify the effect.


In [14]:
# ---
# CHART 10 · Cost-per-Mile vs. Distance -- Efficiency Frontier
# ---

routes["cost_per_mi"] = routes["Total$"] / routes["Dist_mi"]

fig = go.Figure()

for cls in ["Class 8 (16.06 m)", "Class 7 (14.82 m)", "Class 6 (13.98 m)", "Class 5 (11.57 m)"]:
    mask = routes["Truck_Class"] == cls
    sub = routes[mask]
    fig.add_trace(go.Scatter(
        x=sub["Dist_mi"], y=sub["cost_per_mi"],
        mode="markers",
        marker=dict(size=14, color=class_colours[cls],
                    line=dict(width=1.5, color="white"),
                    opacity=0.85),
        name=cls,
        hovertemplate=(
            "<b>Route %{customdata}</b><br>"
            "Distance: %{x:,.0f} mi<br>"
            "Cost/mile: $%{y:.2f}<extra></extra>"
        ),
        customdata=sub["Route"],
    ))

# hyperbolic trend (fixed cost amortisation curve)
x_range = np.linspace(300, 6200, 200)
from scipy.optimize import curve_fit
def hyp(x, a, b): return a + b / x
popt, _ = curve_fit(hyp, routes["Dist_mi"], routes["cost_per_mi"])
fig.add_trace(go.Scatter(
    x=x_range, y=hyp(x_range, *popt),
    mode="lines", name=f"Amortisation curve  (a={popt[0]:.2f}, b={popt[1]:.0f})",
    line=dict(color=ADM["crimson"], width=2.5, dash="dash"),
    hoverinfo="skip",
))

# annotate the 3 cheapest and 3 most expensive routes
cpm_sorted = routes.sort_values("cost_per_mi")
outliers = pd.concat([cpm_sorted.head(3), cpm_sorted.tail(3)])
for _, row_data in outliers.iterrows():
    fig.add_annotation(
        x=row_data["Dist_mi"], y=row_data["cost_per_mi"],
        text=f"R{row_data['Route']}", showarrow=True,
        arrowhead=0, arrowwidth=1, arrowcolor=ADM["navy"],
        ax=0, ay=-25, font=dict(size=10, color=ADM["navy"]),
    )

fig.update_layout(
    title="<b>Economies of Distance</b><br>"
          "<sup>Cost per mile decreases with route length -- fixed costs amortised over more miles</sup>",
    xaxis=dict(title="Route Distance (miles)"),
    yaxis=dict(title="Cost per Mile ($)", tickprefix="$"),
    height=500, width=900,
    legend=dict(orientation="h", yanchor="bottom", y=-0.18, xanchor="center", x=0.5),
    margin=dict(l=60, r=30, t=120, b=90),
)
fig.show()


---
## 11 - HOS Compliance and Rest Break Impact

After 14 consecutive hours on duty, FMCSA rules require a 10-hour mandatory rest. The stacked bar below breaks each route into its driving and resting components. Routes that trip rest breaks are the ones blowing up the schedule -- and the budget.


In [15]:
# ---
# CHART 11 · HOS Impact -- Stacked Bar: Drive vs Rest hours
# ---

# compute rest hours per route from timeline
rest_df = timeline[timeline["Activity"] == "mandatory_rest"].copy()
rest_df["Arrive"] = pd.to_datetime(rest_df["Arrive"])
rest_df["Depart"] = pd.to_datetime(rest_df["Depart"])
rest_df["rest_hrs"] = (rest_df["Depart"] - rest_df["Arrive"]).dt.total_seconds() / 3600
rest_by_route = rest_df.groupby("Route")["rest_hrs"].sum().reset_index()

hos_df = routes[["Route", "Drive_hrs", "Work_hrs", "HOS_break"]].copy()
hos_df = hos_df.merge(rest_by_route, on="Route", how="left")
hos_df["rest_hrs"] = hos_df["rest_hrs"].fillna(0)
hos_df = hos_df.sort_values("Drive_hrs", ascending=True)

fig = go.Figure()
fig.add_trace(go.Bar(
    y="R" + hos_df["Route"].astype(str),
    x=hos_df["Drive_hrs"], orientation="h",
    name="Driving", marker_color=ADM["navy"],
    hovertemplate="Driving: %{x:.1f}h<extra></extra>",
))
fig.add_trace(go.Bar(
    y="R" + hos_df["Route"].astype(str),
    x=hos_df["rest_hrs"], orientation="h",
    name="Mandatory Rest", marker_color=ADM["slate"],
    hovertemplate="Rest: %{x:.1f}h<extra></extra>",
))

fig.add_vline(x=14, line_dash="dot", line_color=ADM["crimson"], line_width=2,
              annotation_text="14h HOS limit", annotation_position="top right",
              annotation_font=dict(color=ADM["crimson"], size=10))

fig.update_layout(
    title="<b>Hours of Service Impact</b><br><sup>Driving hours vs mandatory rest per route · Red line = 14h continuous work limit</sup>",
    barmode="stack",
    xaxis=dict(title="Hours"),
    yaxis=dict(title="", tickfont=dict(size=9)),
    height=750, width=900,
    legend=dict(orientation="h", yanchor="top", y=-0.05, xanchor="left", x=0),
)
fig.show()


---
## 12 - Customer Order Heatmap

Which companies are shipping to which cities? Dense clusters in this matrix point to consolidation opportunities -- customers going to the same destinations should be on the same truck.


In [16]:
# ---
# CHART 12 · Customer x Destination Heatmap
# ---

# pivot: company x destination -> total weight
heat = orders.pivot_table(
    index="Company", columns="Destination",
    values="Weight_kg", aggfunc="sum", fill_value=0
)

# keep only destinations with >2 companies for readability
active_dests = heat.columns[heat.astype(bool).sum(axis=0) >= 2]
heat = heat[active_dests]

# sort both axes by total
heat = heat.loc[heat.sum(axis=1).sort_values(ascending=False).index]
heat = heat[heat.sum(axis=0).sort_values(ascending=False).index]

fig = go.Figure(go.Heatmap(
    z=heat.values,
    x=heat.columns.tolist(),
    y=heat.index.tolist(),
    colorscale=[
        [0.0, ADM["cream"]],
        [0.3, ADM["wheat"]],
        [0.6, ADM["gold"]],
        [0.85, ADM["earth"]],
        [1.0, ADM["navy"]],
    ],
    colorbar=dict(title="kg", tickformat=",.0f"),
    hovertemplate="<b>%{y}</b> -> %{x}<br>%{z:,.0f} kg<extra></extra>",
))

fig.update_layout(
    title="<b>Demand Heatmap</b><br><sup>Customer x Destination · Colour = shipment weight (kg)</sup>",
    xaxis=dict(title="", tickangle=45, tickfont=dict(size=9)),
    yaxis=dict(title="", tickfont=dict(size=10), autorange="reversed"),
    height=600, width=1050,
)
fig.show()


---
## 13 - Fleet KPI Scorecards

The six numbers that matter most. If a dispatcher has 10 seconds to assess fleet performance, this is what they should see.


In [17]:
# ---
# CHART 13 · KPI Indicator Cards
# ---

fig = go.Figure()

kpis = [
    ("Total Cost",      "$126,854",  "dollar",  ADM["navy"]),
    ("Routes",          "33",        "truck",   ADM["gold"]),
    ("Orders Fulfilled","300",       "package", ADM["green"]),
    ("Distance",        "87,391 mi", "road",    ADM["sky"]),
    ("CO2 Emissions",    "162 tonnes","leaf",    ADM["teal"]),
    ("Cost / Mile",     "$1.45",     "chart",   ADM["earth"]),
]

for i, (label, value, _icon, colour) in enumerate(kpis):
    fig.add_trace(go.Indicator(
        mode="number",
        value=None,
        title=dict(text=f"<b>{label}</b>", font=dict(size=14, color=ADM["dark_text"])),
        number=dict(font=dict(size=36, color=colour), valueformat=""),
        domain=dict(
            x=[i/6 + 0.01, (i+1)/6 - 0.01],
            y=[0.0, 1.0],
        ),
    ))

# Use annotations for the values since Indicator number doesn't handle strings well
fig = go.Figure()
for i, (label, value, _icon, colour) in enumerate(kpis):
    x_center = (i + 0.5) / 6
    # title
    fig.add_annotation(
        x=x_center, y=0.72, xref="paper", yref="paper",
        text=f"<b>{label}</b>", showarrow=False,
        font=dict(size=12, color=ADM["slate"]),
    )
    # value
    fig.add_annotation(
        x=x_center, y=0.30, xref="paper", yref="paper",
        text=f"<b>{value}</b>", showarrow=False,
        font=dict(size=26, color=colour),
    )
    # divider line (except last)
    if i < len(kpis) - 1:
        fig.add_shape(
            type="line",
            x0=(i+1)/6, x1=(i+1)/6, y0=0.1, y1=0.9,
            xref="paper", yref="paper",
            line=dict(color=ADM["light_grey"], width=1),
        )

fig.update_layout(
    title="<b>Fleet Performance at a Glance</b><br><sup>ALNS Optimisation · 33 routes · February 2026</sup>",
    height=200, width=1400,
    xaxis=dict(visible=False), yaxis=dict(visible=False),
    margin=dict(l=40, r=40, t=80, b=10),
)
fig.show()


---
## 14 - Fuel Consumption Treemap

A treemap groups fuel burn by driver and route. The area of each block is proportional to gallons burned; darker shading means higher fuel cost. Spot the most expensive driver-route combinations at a glance.


In [18]:
# ---
# CHART 14 · Fuel Consumption Treemap
# ---

tree_df = routes[["Route", "Driver", "Fuel_gal", "Fuel$", "Dist_mi"]].copy()
tree_df["label"] = "R" + tree_df["Route"].astype(str) + " (" + tree_df["Dist_mi"].apply(lambda v: f"{v:,.0f} mi") + ")"

fig = px.treemap(
    tree_df, path=["Driver", "label"],
    values="Fuel_gal", color="Fuel$",
    color_continuous_scale=[
        [0.0, ADM["wheat"]],
        [0.5, ADM["gold"]],
        [1.0, ADM["navy"]],
    ],
    hover_data={"Fuel_gal": ":.0f", "Fuel$": ":$.0f", "Dist_mi": ":,.0f"},
)
fig.update_layout(
    title="<b>Fuel Consumption by Driver & Route</b><br><sup>Area = gallons burned · Colour = fuel cost</sup>",
    height=550, width=1000,
    coloraxis_colorbar=dict(title="Fuel $", tickprefix="$"),
)
fig.update_traces(textinfo="label+value", texttemplate="%{label}<br>%{value:.0f} gal")
fig.show()


---
## 15 - Stops per Route Distribution

The solver caps routes at 8 delivery stops. Single-stop routes are long-haul point-to-point runs; multi-stop routes consolidate nearby destinations. The distribution tells us how aggressively the solver is batching orders.


In [19]:
# ---
# CHART 15 · Stops Distribution -- Styled Histogram
# ---

stop_counts = routes["Stops"].value_counts().sort_index()

fig = go.Figure()
bar_colours = [ADM["navy"] if s == stop_counts.idxmax() else ADM["gold"] for s in stop_counts.index]

fig.add_trace(go.Bar(
    x=stop_counts.index.astype(str),
    y=stop_counts.values,
    marker_color=bar_colours,
    text=stop_counts.values, textposition="outside",
    textfont=dict(size=14, color=ADM["navy"]),
    hovertemplate="%{x} stops: %{y} routes<extra></extra>",
))

# annotate the mode
mode_stops = stop_counts.idxmax()
fig.add_annotation(
    x=str(mode_stops), y=stop_counts.max() + 1,
    text=f"Mode: {mode_stops} stop{'s' if mode_stops > 1 else ''}",
    showarrow=True, arrowhead=2, arrowcolor=ADM["crimson"],
    font=dict(color=ADM["crimson"], size=12),
)

fig.update_layout(
    title="<b>Route Complexity Distribution</b><br><sup>Number of delivery stops per route · Max allowed: 8</sup>",
    xaxis=dict(title="Stops per Route"),
    yaxis=dict(title="Number of Routes"),
    height=380, width=700,
    showlegend=False,
)
fig.show()


---
## 16 - Cost Flow Sankey Diagram

The Sankey traces every dollar from individual routes through the three cost components (fuel, labour, maintenance) into the fleet total. Link width is proportional to cost, so the thickest streams are immediately obvious. This is the best single chart for understanding where the money actually goes.


In [20]:
# ---
# CHART 16 · Cost Flow Sankey Diagram
# ---

def _hex_rgba(hex_color, alpha=0.35):
    h = hex_color.lstrip("#")
    r, g, b = int(h[0:2], 16), int(h[2:4], 16), int(h[4:6], 16)
    return f"rgba({r},{g},{b},{alpha})"

route_labels = ["R" + str(r) + " (" + d.split()[0] + ")"
                for r, d in zip(routes["Route"], routes["Driver"])]
cat_labels = ["Fuel", "Labour", "Maintenance"]
total_label = "Fleet Total"
labels = route_labels + cat_labels + [total_label]
n_r = len(routes)

node_colours = [ADM_SEQ[i % len(ADM_SEQ)] for i in range(n_r)]
node_colours += [ADM["navy"], ADM["gold"], ADM["green"], ADM["crimson"]]

sources, targets, values, link_colours = [], [], [], []
for i, (_, row) in enumerate(routes.iterrows()):
    for off, col_name, rgba_col in [
        (0, "Fuel$",   _hex_rgba(ADM["navy"])),
        (1, "Labour$", _hex_rgba(ADM["gold"])),
        (2, "Maint$",  _hex_rgba(ADM["green"])),
    ]:
        sources.append(i)
        targets.append(n_r + off)
        values.append(row[col_name])
        link_colours.append(rgba_col)

# Category -> fleet total
for off, total_val, rgba_col in [
    (0, routes["Fuel$"].sum(),   _hex_rgba(ADM["navy"], 0.5)),
    (1, routes["Labour$"].sum(), _hex_rgba(ADM["gold"], 0.5)),
    (2, routes["Maint$"].sum(),  _hex_rgba(ADM["green"], 0.5)),
]:
    sources.append(n_r + off)
    targets.append(n_r + 3)
    values.append(total_val)
    link_colours.append(rgba_col)

fig = go.Figure(go.Sankey(
    arrangement="snap",
    node=dict(
        pad=20, thickness=22,
        label=labels, color=node_colours,
        line=dict(color=ADM["cream"], width=0.5),
        hovertemplate="<b>%{label}</b><br>$%{value:,.0f}<extra></extra>",
    ),
    link=dict(
        source=sources, target=targets, value=values,
        color=link_colours,
        hovertemplate="%{source.label} -> %{target.label}<br>$%{value:,.0f}<extra></extra>",
    ),
))
fig.update_layout(
    title="<b>Cost Flow: Routes -> Components -> Fleet Total</b><br>"
          "<sup>Link width proportional to cost</sup>",
    height=520, width=1100,
    font=dict(size=11, color=ADM["dark_text"]),
    margin=dict(l=10, r=10, t=80, b=10),
)
fig.show()


---
## 17 - Efficiency Frontier

The sweet spot for any route is the lower-left corner of this scatter: more deliveries per mile at lower cost per kilogram. Bubble size reflects the number of stops; colour reflects capacity utilisation. The dashed green line is the efficient frontier -- routes that sit on this boundary are Pareto-optimal for unit economics. Everything above the line has room to improve.


In [21]:
# ---
# CHART 17 · Efficiency Frontier
# ---

# Build per-route metrics
class_capacity = {
    "Class 8 (16.06 m)": 10000,
    "Class 7 (14.82 m)":  7500,
    "Class 6 (13.98 m)":  6000,
    "Class 5 (11.57 m)":  4000,
}

cost_per_kg = routes["Total$"] / routes["Weight_kg"]
dist_per_stop = routes["Dist_mi"] / routes["Stops"]
util_pct = (routes["Weight_kg"] / routes["Truck_Class"].map(class_capacity) * 100).clip(upper=100)

# Lower convex hull using Andrew's monotone-chain algorithm
pts = sorted(set(zip(dist_per_stop, cost_per_kg)), key=lambda p: (p[0], p[1]))

def _lower_hull(points):
    hull = []
    for p in points:
        while len(hull) >= 2:
            o, a, b = hull[-2], hull[-1], p
            cross = (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
            if cross <= 0:
                hull.pop()
            else:
                break
        hull.append(p)
    return hull

hull = _lower_hull(pts)
hull_x = [p[0] for p in hull]
hull_y = [p[1] for p in hull]

fig = go.Figure()

if len(hull_x) > 1:
    fig.add_trace(go.Scatter(
        x=hull_x, y=hull_y, mode="lines",
        line=dict(color=ADM["green"], width=2, dash="dash"),
        name="Efficient Frontier", hoverinfo="skip",
    ))
    fig.add_trace(go.Scatter(
        x=hull_x, y=hull_y, mode="lines", line=dict(width=0),
        fill="tozeroy", fillcolor=_hex_rgba(ADM["green"], 0.08),
        showlegend=False, hoverinfo="skip",
    ))

fig.add_trace(go.Scatter(
    x=dist_per_stop, y=cost_per_kg, mode="markers",
    marker=dict(
        size=routes["Stops"] * 5 + 8,
        color=util_pct,
        colorscale=[[0, ADM["crimson"]], [0.5, ADM["gold"]], [1.0, ADM["green"]]],
        showscale=True,
        colorbar=dict(title="Util %", ticksuffix="%", thickness=12, len=0.6),
        line=dict(width=1, color="rgba(0,38,58,0.25)"),
        opacity=0.88,
    ),
    text=[
        f"<b>Route {r}</b><br>{d}<br>Cost/kg: ${cpk:.2f}<br>"
        f"Dist/stop: {dps:.0f} mi<br>Util: {u:.0f}%"
        for r, d, cpk, dps, u in zip(
            routes["Route"], routes["Driver"],
            cost_per_kg, dist_per_stop, util_pct
        )
    ],
    hoverinfo="text", name="Routes",
))

fig.update_layout(
    title="<b>Route Efficiency Frontier</b><br>"
          "<sup>Cost per kg vs distance per stop · Below the dashed line = most efficient</sup>",
    xaxis_title="Distance per Stop (mi)",
    yaxis_title="Cost per kg ($)",
    height=450, width=900,
    legend=dict(orientation="h", y=1.08),
)
fig.show()


---
## 18 - Cumulative Delivery Timeline

A step chart of deliveries accumulating over time. Steep sections mean trucks are completing stops in quick succession; flat plateaus mean nobody is delivering -- typically overnight rest periods or gaps between route departures. Milestone annotations mark the 25%, 50%, 75%, and 100% completion points.


In [22]:
# ---
# CHART 18 · Cumulative Delivery Timeline
# ---

# Parse delivery events from the timeline CSV
delivery_events = timeline[timeline["Activity"] == "delivery"].copy()
delivery_events["Arrive"] = pd.to_datetime(delivery_events["Arrive"])
delivery_events = delivery_events.sort_values("Arrive").reset_index(drop=True)
delivery_events["cum"] = range(1, len(delivery_events) + 1)

fig = go.Figure()

# Shaded area
fig.add_trace(go.Scatter(
    x=delivery_events["Arrive"], y=delivery_events["cum"],
    mode="lines", line=dict(width=0),
    fill="tozeroy", fillcolor=_hex_rgba(ADM["navy"], 0.08),
    showlegend=False, hoverinfo="skip",
))

# Step line with markers
fig.add_trace(go.Scatter(
    x=delivery_events["Arrive"], y=delivery_events["cum"],
    mode="lines+markers",
    line=dict(shape="hv", width=2.5, color=ADM["navy"]),
    marker=dict(size=5, color=ADM["gold"], line=dict(width=1, color=ADM["navy"])),
    text=[
        f"<b>{row.City}</b><br>{row.Driver} (R{row.Route})<br>"
        f"Delivery #{row.cum}<br>{row.Arrive.strftime('%b %d %H:%M')}"
        for _, row in delivery_events.iterrows()
    ],
    hoverinfo="text", name="Deliveries",
))

# Milestone annotations at 25%, 50%, 75%, 100%
total = len(delivery_events)
for pct in [0.25, 0.50, 0.75, 1.0]:
    idx = min(int(total * pct) - 1, total - 1)
    if idx < 0:
        continue
    row = delivery_events.iloc[idx]
    fig.add_annotation(
        x=row["Arrive"], y=row["cum"],
        text=f"{int(pct * 100)}%", showarrow=True, arrowhead=2,
        ax=0, ay=-28,
        font=dict(size=9, color=ADM["crimson"]),
        arrowcolor=ADM["crimson"], arrowwidth=1.5,
        bgcolor="rgba(255,255,255,0.85)",
        bordercolor=ADM["crimson"], borderwidth=1, borderpad=3,
    )

fig.update_layout(
    title="<b>Cumulative Delivery Timeline</b><br>"
          "<sup>Step chart showing dispatch cadence across the fleet plan</sup>",
    xaxis=dict(title="Time", type="date"),
    yaxis_title="Cumulative Deliveries",
    height=420, width=1000,
    showlegend=False,
)
fig.show()


---
## 19 - Monte Carlo Risk Analysis (Multi-Solver)

We ran 1,000 Monte Carlo trials for each solver under randomised conditions: fuel prices fluctuating +/-15%, travel times varying +/-20%, and a 5% truck breakdown probability per route.

The violin plot shows the full shape of each solver's cost distribution -- wider sections mean more probability mass at that cost level. The box-plot internals mark the median and interquartile range.



In [23]:
# ---
# CHART 19a · Multi-Solver MC Cost Violin
# CHART 19b · Cost per Mile Box Plot
# ---
import pickle

cache_path = RESULTS / "solver_cache.pkl"
if cache_path.exists():
    with open(cache_path, "rb") as f:
        solver_data = pickle.load(f)
    print(f"Loaded cached results for: {', '.join(s.upper() for s in solver_data)}")
else:
    solver_data = {}
    print("No solver cache found -- run `python main.py --solver all --dashboard` first")

# ---- 19a: MC Violin ----
if solver_data:
    palette = [ADM["green"], ADM["gold"], ADM["navy"], ADM["crimson"], ADM["sky"], ADM["teal"]]
    fig = go.Figure()
    idx = 0
    for sname, sdata in solver_data.items():
        mc = sdata.get("mc_result")
        if mc is None or not mc.trial_costs:
            continue
        col = palette[idx % len(palette)]
        fig.add_trace(go.Violin(
            y=mc.trial_costs, name=sname.upper(),
            box_visible=True, meanline_visible=True,
            fillcolor=_hex_rgba(col, 0.25), line_color=col,
            marker=dict(color=col, size=2, opacity=0.15),
            points="outliers", spanmode="hard",
        ))
        fig.add_annotation(
            x=sname.upper(), y=mc.cost_p50,
            text=f"P50: ${mc.cost_p50:,.0f}",
            showarrow=True, arrowhead=0, ax=60, ay=0,
            font=dict(size=9, color=col), arrowcolor=col,
        )
        idx += 1

    fig.update_layout(
        title="<b>Monte Carlo Cost Risk Distribution by Strategy</b><br>"
              "<sup>1,000 trials per solver · violin width = probability density</sup>",
        yaxis_title="Simulated Total Cost ($)",
        yaxis_tickprefix="$",
        height=480, width=900,
        showlegend=False, violingap=0.35,
    )
    fig.show()
else:
    print("Skipping MC violin -- no solver cache")

Loaded cached results for: GREEDY, COLGEN, ALNS


**Reading the cost-per-mile chart:** The high-CPM outliers (top of chart) are **tiny bubbles** -- they represent very short nearby deliveries (under 10 miles) where fixed labour and maintenance costs get divided by very few miles, inflating the rate. These routes are cheap in absolute dollars (under $50 each). The **fleet-weighted average** annotations show the real per-mile economics: all three strategies run at roughly $1.45-1.47/mi. ALNS saves money not by lowering the per-mile rate, but by **driving 8,200 fewer total miles** through better stop consolidation (3.0 stops/route vs Greedy's 1.8).

In [24]:
# ---
# CHART 19b · Cost per Mile by Strategy
# Bubble size = route distance so viewers see that high-CPM outliers are tiny routes
# ---

if solver_data:
    palette = [ADM["green"], ADM["gold"], ADM["navy"], ADM["crimson"], ADM["sky"], ADM["teal"]]
    fig = go.Figure()

    for idx, (sname, sdata) in enumerate(solver_data.items()):
        costs = sdata["costs"]
        cpm = []
        dist = []
        total_cost_sum = 0
        total_dist_sum = 0
        for c in costs:
            if c["distance_mi"] > 0:
                cpm.append(c["total_cost"] / c["distance_mi"])
                dist.append(c["distance_mi"])
                total_cost_sum += c["total_cost"]
                total_dist_sum += c["distance_mi"]

        fleet_avg_cpm = total_cost_sum / total_dist_sum if total_dist_sum else 0
        col = palette[idx % len(palette)]

        # Bubble size proportional to route distance (longer routes = bigger bubbles)
        max_dist = max(dist) if dist else 1
        sizes = [max(5, d / max_dist * 20) for d in dist]

        fig.add_trace(go.Box(
            y=cpm, name=sname.upper(),
            boxpoints=False,
            line=dict(color=col, width=2),
            fillcolor=_hex_rgba(col, 0.15),
            hoverinfo="y",
        ))

        # Overlay scatter with sized points
        fig.add_trace(go.Scatter(
            x=[sname.upper()] * len(cpm),
            y=cpm, mode="markers",
            marker=dict(color=col, size=sizes, opacity=0.5,
                        line=dict(width=0.5, color=ADM["navy"])),
            hovertemplate=(
                f"<b>{sname.upper()}</b><br>"
                "CPM: $%{y:.2f}/mi<br>"
                "Route dist: %{text} mi<extra></extra>"
            ),
            text=[f"{d:.0f}" for d in dist],
            showlegend=False,
        ))

        # Fleet-weighted average line
        fig.add_annotation(
            x=sname.upper(), y=fleet_avg_cpm,
            text=f"Fleet avg: ${fleet_avg_cpm:.2f}/mi",
            showarrow=True, arrowhead=0, ax=80, ay=0,
            font=dict(size=10, color=col, family="Segoe UI"),
            arrowcolor=col, arrowwidth=1.5,
        )

    fig.update_layout(
        title="<b>Cost per Mile by Strategy</b><br>"
              "<sup>Bubble size = route distance · Small bubbles at the top are short nearby runs "
              "where fixed costs dominate</sup>",
        yaxis_title="Cost per Mile ($/mi)", yaxis_tickprefix="$",
        height=480, width=900, showlegend=False,
    )
    fig.show()
else:
    print("Skipping cost-per-mile chart -- no solver cache")


---
## 20 - Multi-Solver Risk Metrics Summary

The table below puts exact numbers on the Monte Carlo results. Deterministic cost is the "perfect information" baseline. P50 and P95 show what happens under real-world randomness. The P95-P5 spread measures how wide the cost distribution gets -- a solver with a narrow spread is more predictable, which is what operations teams actually care about.


In [25]:
# ---
# CHART 20 · Multi-Solver MC Risk Summary Table
# ---

if solver_data:
    headers = ["Metric"]
    rows_data = {
        "Deterministic Cost": [],
        "P50 (Median)": [],
        "P95 (Worst Case)": [],
        "Robustness Score": [],
        "P95-P5 Spread": [],
        "CO2 P50 (kg)": [],
    }
    for sname, sdata in solver_data.items():
        mc = sdata.get("mc_result")
        headers.append(sname.upper())
        if mc:
            rows_data["Deterministic Cost"].append(f"${mc.deterministic_cost:,.0f}")
            rows_data["P50 (Median)"].append(f"${mc.cost_p50:,.0f}")
            rows_data["P95 (Worst Case)"].append(f"${mc.cost_p95:,.0f}")
            rows_data["Robustness Score"].append(f"{mc.robustness_score:.0f}%")
            rows_data["P95-P5 Spread"].append(f"${mc.cost_p95 - mc.cost_p5:,.0f}")
            rows_data["CO2 P50 (kg)"].append(f"{mc.co2_p50:,.0f}")
        else:
            for k in rows_data:
                rows_data[k].append("--")

    cell_values = [[k] + v for k, v in rows_data.items()]
    # Transpose for go.Table (each sub-list is a column)
    n_cols = len(headers)
    columns = [[] for _ in range(n_cols)]
    for row in cell_values:
        for ci, val in enumerate(row):
            columns[ci].append(val)

    fig = go.Figure(go.Table(
        header=dict(
            values=[f"<b>{h}</b>" for h in headers],
            fill_color=ADM["navy"],
            font=dict(color=ADM["cream"], size=17, family="Segoe UI"),
            align=["left"] + ["center"] * (n_cols - 1),
            height=52, line=dict(width=0),
        ),
        cells=dict(
            values=columns,
            fill_color=[[ADM["cream"], "white"] * 3],
            font=dict(color=ADM["dark_text"], size=16, family="Segoe UI"),
            align=["left"] + ["center"] * (n_cols - 1),
            height=44, line=dict(width=0.5, color=ADM["light_grey"]),
        ),
    ))
    fig.update_layout(
        title=dict(
            text="<b>Monte Carlo Risk Metrics -- Strategy Comparison</b>",
            font=dict(size=20),
        ),
        height=460, width=1100,
        margin=dict(l=5, r=5, t=80, b=10),
    )
    fig.show()
else:
    print("Skipping MC summary table -- no solver cache")



---
## 21 - Solver Comparison: Which Algorithm Wins?

We built four solvers, each with a different philosophy. The practical question is: which one should run in production?

### The Four Solvers

| Solver | Approach | Mathematical Class |
|:---|:---|:---|
| Greedy | Always picks the nearest unvisited stop | Constructive heuristic (nearest-insertion) |
| Column Generation | Routes "bid" to serve orders via LP pricing | LP relaxation + Dantzig-Wolfe decomposition |
| ALNS | Iteratively destroys and rebuilds parts of the solution, accepting some worse moves to escape local optima | Adaptive metaheuristic with simulated annealing |
| Pareto | Optimises cost, CO2, and time simultaneously, returning a set of non-dominated solutions | Multi-objective optimisation (non-dominated sorting) |

### Why There Is No Universal Best

This is a known result in combinatorial optimisation: the No Free Lunch Theorem (Wolpert and Macready, 1997) states that no algorithm dominates all others across every possible problem instance. But for our specific structure -- single depot, heterogeneous fleet, HOS constraints -- we can rank them empirically using a composite score:

$$\text{Value Score} = \frac{\text{Solution Quality (lower cost)}}{\text{Computational Budget (runtime)}} \times \text{Robustness (MC P95/P50 ratio)}$$

### Deterministic vs Stochastic Assessment

The charts below compare all four solvers on two dimensions:

1. Deterministic cost -- the solution assuming perfect information about fuel prices, travel times, and truck availability
2. Monte Carlo robustness -- how the solution holds up when we inject real-world noise (fuel volatility, breakdowns, travel delays)

A solver that finds a cheap plan but crumbles under uncertainty is less useful than one that costs slightly more but performs reliably under stress.



In [26]:

# ---
# CHART 21 · Multi-Solver Comparison Dashboard
# ---

if solver_data:
    solver_names = [s.upper() for s in solver_data]
    palette_s = [ADM["green"], ADM["gold"], ADM["navy"], ADM["crimson"],
                 ADM["sky"], ADM["teal"]]

    # -- Extract metrics --
    det_costs, p50s, p95s, robustness = [], [], [], []
    # Known runtimes (from experiments; store in future runs)
    runtime_map = {"greedy": 0.03, "colgen": 42, "alns": 600, "pareto": 120}

    for sname, sdata in solver_data.items():
        mc = sdata.get("mc_result")
        det_costs.append(mc.deterministic_cost if mc else sum(c["total_cost"] for c in sdata["costs"]))
        p50s.append(mc.cost_p50 if mc else 0)
        p95s.append(mc.cost_p95 if mc else 0)
        robustness.append(mc.robustness_score if mc else 0)

    runtimes = [runtime_map.get(s, 60) for s in solver_data]

    # ---- Chart 21a: Deterministic Cost Comparison (bar chart) ----
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Deterministic Fleet Cost", "Runtime vs Solution Quality"),
        horizontal_spacing=0.15,
        specs=[[{"type": "bar"}, {"type": "scatter"}]],
    )

    # Bar chart: deterministic cost per solver
    min_cost = min(det_costs)
    bar_colours = [ADM["navy"] if c == min_cost else ADM["slate"] for c in det_costs]
    fig.add_trace(go.Bar(
        x=solver_names, y=det_costs,
        marker_color=bar_colours,
        text=[f"${c:,.0f}" for c in det_costs],
        textposition="outside", textfont=dict(size=11),
        hovertemplate="<b>%{x}</b><br>$%{y:,.0f}<extra></extra>",
        showlegend=False,
    ), row=1, col=1)

    # Star annotation on best solver
    best_idx = det_costs.index(min_cost)
    fig.add_annotation(
        x=solver_names[best_idx], y=min_cost,
        text="* Best", showarrow=True, arrowhead=2,
        ax=0, ay=-30, font=dict(size=10, color=ADM["crimson"]),
        arrowcolor=ADM["crimson"],
        row=1, col=1,
    )

    # Scatter: runtime vs cost (bigger = worse robustness)
    for i, sname in enumerate(solver_names):
        fig.add_trace(go.Scatter(
            x=[runtimes[i]], y=[det_costs[i]],
            mode="markers+text",
            marker=dict(size=max(8, 40 - robustness[i] * 0.3),
                        color=palette_s[i % len(palette_s)],
                        line=dict(width=1.5, color="white")),
            text=[sname], textposition="top center",
            textfont=dict(size=10),
            hovertemplate=(
                f"<b>{sname}</b><br>"
                f"Runtime: {runtimes[i]:.1f}s<br>"
                f"Cost: ${det_costs[i]:,.0f}<br>"
                f"Robustness: {robustness[i]:.0f}%"
                "<extra></extra>"
            ),
            showlegend=False,
        ), row=1, col=2)

    fig.update_xaxes(title_text="Solver", row=1, col=1)
    fig.update_yaxes(title_text="Total Cost ($)", tickprefix="$", row=1, col=1)
    fig.update_xaxes(title_text="Runtime (seconds)", type="log", row=1, col=2)
    fig.update_yaxes(title_text="Total Cost ($)", tickprefix="$", row=1, col=2)

    fig.update_layout(
        title="<b>Solver Performance Comparison</b><br>"
              "<sup>Which algorithm delivers the best cost for the computational budget?</sup>",
        height=450, width=1050,
    )
    fig.show()

    # ---- Chart 21b: Risk-Adjusted Radar ----
    if all(p95s):
        # Normalise each metric to 0-100 (higher = better)
        max_cost = max(det_costs) * 1.05
        max_p95 = max(p95s) * 1.05
        max_rt = max(runtimes) * 1.05

        categories = ["Low Cost", "Robustness", "Speed", "Low P95 Risk", "Consistency"]

        fig2 = go.Figure()
        for i, sname in enumerate(solver_data):
            mc = solver_data[sname].get("mc_result")
            spread = (mc.cost_p95 - mc.cost_p5) if mc else 1
            max_spread = max((solver_data[s].get("mc_result").cost_p95 -
                              solver_data[s].get("mc_result").cost_p5)
                             for s in solver_data if solver_data[s].get("mc_result")) * 1.05

            vals = [
                100 * (1 - det_costs[i] / max_cost),       # low cost
                robustness[i],                               # robustness score
                100 * (1 - runtimes[i] / max_rt),           # speed
                100 * (1 - p95s[i] / max_p95),              # low P95 risk
                100 * (1 - spread / max_spread),             # consistency
            ]
            vals.append(vals[0])  # close the polygon
            cats = categories + [categories[0]]

            col = palette_s[i % len(palette_s)]
            fig2.add_trace(go.Scatterpolar(
                r=vals, theta=cats,
                fill="toself",
                fillcolor=_hex_rgba(col, 0.12),
                line=dict(color=col, width=2),
                name=sname.upper(),
                hovertemplate="<b>%{theta}</b>: %{r:.0f}/100<extra></extra>",
            ))

        fig2.update_layout(
            title="<b>Multi-Dimensional Solver Scorecard</b><br>"
                  "<sup>Radar chart: larger area = better overall performance</sup>",
            polar=dict(
                bgcolor=ADM["cream"],
                radialaxis=dict(visible=True, range=[0, 100],
                                ticksuffix="", gridcolor=ADM["light_grey"]),
                angularaxis=dict(tickfont=dict(size=11)),
            ),
            height=520, width=700,
            legend=dict(orientation="h", y=-0.1, x=0.5, xanchor="center"),
        )
        fig2.show()
else:
    print("Solver comparison requires solver_cache.pkl -- run `python main.py --solver all`")


### Cost Breakdown by Component -- Where Does Each Dollar Go?

The bar chart above shows total cost, but the real question for operations is: which cost lever moves the needle the most? The chart below decomposes each solver's total into fuel, labour, and maintenance. ALNS reduces **every** component -- fuel drops from $59.6k to $54.0k, labour from $54.2k to $50.5k, and maintenance from $13.1k to $11.9k. The savings are proportional across all three, which tells us the improvement comes from shorter total distance (fewer miles driven), not from structural changes like fewer trucks or different crew schedules.

In [27]:
# ---
# CHART 22 · Cost Components by Strategy (Fuel / Labour / Maintenance)
# Vertical stacked bars, ADM brand colours, compact
# ---

if solver_data:
    solver_names_22 = [s.upper() for s in solver_data]

    fuel_totals, labour_totals, maint_totals = [], [], []
    for sname, sdata in solver_data.items():
        costs_list = sdata["costs"]
        fuel_totals.append(sum(c["fuel_cost"] for c in costs_list))
        labour_totals.append(sum(c["labour_cost"] for c in costs_list))
        maint_totals.append(sum(c["maint_cost"] for c in costs_list))

    totals = [f + l + m for f, l, m in zip(fuel_totals, labour_totals, maint_totals)]

    # Sort by total cost (cheapest first, left to right)
    order = sorted(range(len(totals)), key=lambda i: totals[i])
    s_names  = [solver_names_22[i] for i in order]
    s_fuel   = [fuel_totals[i] for i in order]
    s_labour = [labour_totals[i] for i in order]
    s_maint  = [maint_totals[i] for i in order]
    s_totals = [totals[i] for i in order]

    # ADM brand palette
    adm_navy  = "#00263A"
    adm_gold  = "#D4A843"
    adm_green = "#4A7C59"

    fig = go.Figure()

    fig.add_trace(go.Bar(
        name="Fuel",
        x=s_names, y=s_fuel,
        marker=dict(color=adm_navy),
        text=[f"${v/1000:.0f}k" for v in s_fuel],
        textposition="inside", textfont=dict(size=11, color="white"),
        hovertemplate="<b>%{x}</b> Fuel: $%{y:,.0f}<extra></extra>",
    ))

    fig.add_trace(go.Bar(
        name="Labour",
        x=s_names, y=s_labour,
        marker=dict(color=adm_gold),
        text=[f"${v/1000:.0f}k" for v in s_labour],
        textposition="inside", textfont=dict(size=11, color="#1A1A1A"),
        hovertemplate="<b>%{x}</b> Labour: $%{y:,.0f}<extra></extra>",
    ))

    fig.add_trace(go.Bar(
        name="Maintenance",
        x=s_names, y=s_maint,
        marker=dict(color=adm_green),
        text=[f"${v/1000:.0f}k" for v in s_maint],
        textposition="inside", textfont=dict(size=11, color="white"),
        hovertemplate="<b>%{x}</b> Maint: $%{y:,.0f}<extra></extra>",
    ))

    # Total labels above each bar
    for i, (name, t) in enumerate(zip(s_names, s_totals)):
        fig.add_annotation(
            x=name, y=t + 2000,
            text=f"<b>${t/1000:.0f}k</b>",
            showarrow=False, yanchor="bottom",
            font=dict(size=12, color="#1A1A1A"),
        )

    fig.update_layout(
        barmode="stack",
        title=dict(
            text="<b>Cost Components by Strategy</b>",
            x=0.01, xanchor="left",
            font=dict(size=15, color="#1A1A1A"),
        ),
        font=dict(size=12, color="#333"),
        plot_bgcolor="white",
        paper_bgcolor="white",
        yaxis=dict(
            showgrid=True, gridcolor="#E5E5E5", gridwidth=0.5,
            zeroline=False, showline=False, tickprefix="$",
            tickformat=",", title="",
        ),
        xaxis=dict(
            showgrid=False, showline=False,
            tickfont=dict(size=13, color="#1A1A1A"),
        ),
        legend=dict(
            orientation="h", y=-0.15, x=0.5, xanchor="center",
            font=dict(size=11), bgcolor="rgba(0,0,0,0)",
            traceorder="normal",
        ),
        margin=dict(l=70, r=30, t=50, b=50),
        height=600,
        width=400,
    )
    fig.show()
else:
    print("Skipping cost components chart -- no solver cache")

### Verdict: ALNS is the Production-Grade Choice

| Criterion | Winner | Reasoning |
|:---|:---|:---|
| Lowest cost | ALNS ($116,396) | Saves $10,458 vs Greedy by consolidating stops and cutting 8,225 miles |
| Fastest runtime | Greedy (<1 s) | Instant, but 8.2% more expensive than ALNS |
| Best risk profile | ALNS | Tightest P95 ($126,940) -- still cheaper than Greedy's deterministic cost |
| Multi-objective insight | Pareto | Exposes the cost-CO2-time trade-off frontier |
| Theoretical optimality | Column Generation | LP relaxation provides a provable lower bound |

The practical recommendation: run ALNS for daily batch dispatch (a 10-minute solve is perfectly acceptable as an overnight job). Keep Greedy as a real-time fallback for intra-day re-routing when a truck breaks down or a rush order comes in. Run Pareto once a month for strategic fleet composition decisions -- for instance, whether it makes sense to buy more Class 6 trucks.

---

## Results Synthesis

### Cost Structure
Fuel (46%) and labour (43%) dominate fleet costs. Maintenance is a distant third at 10%. This tells us where optimisation effort pays off the most:
1. Shorter routes -- every 100 miles saved means roughly $56 in fuel and $20 in maintenance
2. Fewer HOS rest breaks -- each avoided 10-hour rest period saves approximately $250 in dead labour time

### Operational Efficiency
- The ALNS solver packs trucks close to capacity while consolidating more stops per route (up to 7 stops vs Greedy's max of 3), cutting total distance from 87,391 miles to 79,166 miles.
- Economies of distance hold up in the data: long-haul routes run about $1.20/mi compared to $1.65/mi for shorter runs. The amortisation curve confirms this is classic fixed-cost dilution.
- Five routes exceed $5,000 each and account for a disproportionate share of total spend. These are the prime candidates for consolidation or shifting to rail intermodal.

### Risk Profile
The ALNS solution is robust under stress. Monte Carlo simulation shows the P95 worst-case cost ($126,940) is only about 9% above the deterministic estimate ($116,396) -- and that worst case is still cheaper than what Greedy costs on a good day. Fuel price volatility is the biggest risk factor. Truck breakdowns have minimal impact because the fleet has built-in multi-truck redundancy.

### Carbon Footprint
The ALNS fleet produces 147 tonnes of CO2 across all routes, down from 162 tonnes under Greedy -- a 10% emissions reduction that comes free with the cost savings. The relationship between distance and emissions is almost perfectly linear (R-squared above 0.99), which means route length is the single best predictor of carbon impact.



---
## Limitations of This Analysis

Every model simplifies reality. The value of listing limitations is not to apologise for the work -- it is to show exactly where the assumptions break down and how to fix them.

### 1. Distance Approximation

We use the Haversine formula (great-circle distance on a sphere) scaled by a 1.30 road factor:

$$d_{\text{road}}(i,j) = 1.30 \times 2R \arcsin\!\left(\sqrt{\sin^2\!\left(\frac{\phi_j - \phi_i}{2}\right) + \cos\phi_i \cos\phi_j \sin^2\!\left(\frac{\lambda_j - \lambda_i}{2}\right)}\right)$$

where $R = 3{,}959$ mi (Earth radius), $\phi$ = latitude, $\lambda$ = longitude.

The 1.30 factor comes from empirical studies (Boscoe et al., 2012) showing that US road distances average 1.25 to 1.40 times the straight-line distance. We use the midpoint.

What this misses: mountain passes, construction zones, toll roads, urban congestion. A production system should call the Google Maps Distance Matrix API or OSRM for actual road-network distances with real-time traffic.

### 2. Single-Depot Assumption

All 33 routes start and end at Cincinnati, OH. This matches The Company's current hub-and-spoke model, but it prevents us from exploring multi-depot routing (for example, adding a western hub in Denver or Dallas), open-ended routes where trucks finish at the last delivery, or cross-docking at intermediate warehouses.

Adding even one western depot could cut total fleet cost by 15-25% by eliminating the longest cross-country legs to Las Vegas, Portland, and Seattle.

### 3. Deterministic Demand

We assume all 300 orders are known upfront. In practice, orders arrive throughout the day, customers change quantities, and agricultural commodities have seasonal peaks. Our Monte Carlo simulation partially addresses this by stress-testing under fuel and travel uncertainty, but a full stochastic VRP formulation would model demand as a Poisson arrival process and re-optimise on rolling horizons.

### 4. Constant Speed Assumption

We assume 55 mph average speed for all trucks. In reality, Class 8 heavy trucks are slower on grades than Class 5 medium trucks, urban segments run 25-35 mph versus 65 mph on interstate highways, and speed varies by time of day and weather. A better model would use speed profiles per road segment from historical telematics data.

### 5. Cost Model Simplifications

Our three-component model (fuel + labour + maintenance) leaves out toll roads ($50-200 per route on eastern turnpikes), insurance and registration (fixed, should be pro-rated per route), cargo insurance (varies by commodity value), opportunity cost (a truck on a 5-day Las Vegas run cannot serve local Cincinnati orders), and driver overtime structures under HOS.

### 6. No Customer Time Windows

The model enforces driver HOS constraints but not customer delivery windows (for example, "deliver between 9 AM and 2 PM"). Adding hard time windows would drastically reduce the feasible solution space, require penalty functions for violations, and better reflect real service level agreements.

---

## Potential Improvements

### Near-Term (next 3 days)

| Improvement | Expected Impact | Effort |
|:---|:---|:---|
| Real road distances (Google/OSRM API) | 5-10% cost accuracy gain | 2-3 days |
| Customer time windows | Better SLA compliance | 1 week |
| Speed profiles by road type | 3-5% routing improvement | 3-4 days |
| Toll-aware routing | $2,000-5,000 savings on avoidance | 2-3 days |



---

## Methodology Summary

| Stage | Method | Key Parameters |
|:---|:---|:---|
| Distance | Haversine x 1.30 road factor | R = 3,959 mi |
| Solver | ALNS metaheuristic | 6,000 iterations, T0 = 500, cooling = 0.9997 |
| Destroy operators | Random, worst, Shaw, route-remove | 10-35% removal per iteration |
| Repair operators | Greedy, regret-2, perturbed | Regret-2 dominated at 54% selection |
| Cost model | Fuel + labour + maintenance | $3.75/gal, $0.15/mi maintenance |
| HOS compliance | FMCSA rules | 14h work window / 10h mandatory rest |
| Emissions | EPA diesel emission factor | 10.18 kg CO2 per gallon |
| Risk simulation | Monte Carlo with 1,000 trials | Fuel +/-15%, travel +/-20%, breakdown 5% |

---
